In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision 
from torchvision import transforms

from torch.utils.data import Dataset, DataLoader
import random
import torchtext.data as textdata

from tqdm.autonotebook import tqdm

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from google.colab import drive
import pandas as pd
import string
import nltk
import torchtext
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
import re
import spacy
nlp = spacy.load('en')

from sklearn.metrics import accuracy_score

import time
import sys, os

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu") 

In [ ]:
#HYPERPARAMETERS
#optimizer = torch.optim.Adam(model.parameters(), lr = 2e-4,weight_decay=1e-5)
criterion = nn.BCEWithLogitsLoss()
vocab_Max = 50000

In [ ]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/My\ Drive/Colab Notebooks

/content/gdrive/My Drive/Colab Notebooks


In [ ]:
New_Process_Data = True

In [ ]:
df = pd.read_csv('train.csv')
#test_df = pd.read_csv('test.csv')
#test_labels_df = pd.read_csv('test_labels.csv')
process_train_path = 'processtrain.csv'
process_test_path = 'processtest.csv'

In [ ]:
#print('Null values in training set?')
#print(df.isnull().sum())
#print('')
#print('Null values in test set?')
#print(test_df.isnull().sum())

In [ ]:
#df['toxic'].value_counts()

In [ ]:
#df['obscene'].value_counts()

In [ ]:
#df['comment_text'][36]

In [ ]:
#Inspiration for this function gotten from https://www.kaggle.com/prabhatkumarsahu/toxic-comment-classification-thecaffeinedev
punctuations = string.punctuation
stopwords_list = stopwords.words("english")
spacy_tokenizer = torchtext.data.utils.get_tokenizer('spacy')
lemmatizer = WordNetLemmatizer()
def processing(text):
  
    def tokenizer(text):
        text = str.split(text)
        return text
    
    def remove_punctuations(sentence):
        result = "".join([i if i not in punctuations and not i.isdigit() else " " for i in sentence])
        return result
    
    def word_lemmatizer(sentence):
        result = lemmatizer.lemmatize(sentence)
        return result
    
    def word_lowercase(sentence):
        return sentence.lower()
    
    def remove_URL(text):
        url = re.compile(r'https?://\S+|www\.\S+')
        html = re.compile(r'<.*?>')
        text = html.sub(r'',text)
        text = url.sub(r'',str(text))
        return text
  
    def remove_newline(text):
        return text.rstrip("\n")
    
    def clean_comment(sentence):
        result = []
        sentence = remove_newline(sentence)
        sentence = remove_URL(sentence)
        sentence = word_lowercase(sentence)
        sentence = word_lemmatizer(sentence)
        sentence = remove_punctuations(sentence)
        sentence = tokenizer(sentence)

        result = " ".join(sentence)
        return result
     
    text = clean_comment(text)
    if text == "":
        text = "None"
    return text

In [ ]:
import csv
from tqdm import tqdm

if New_Process_Data:
    with open('train.csv', "r", encoding="utf8") as in_csv, open('processtrain.csv', "w", newline="", encoding="utf8") as out_csv:
        read = csv.reader(in_csv)
        write = csv.writer(out_csv)
        next(read, None) # Skip header
        for i in tqdm(read):
            i[1] = processing(i[1])
            try:
                write.writerow(i)
            except Exception as e:
                print(e)

159571it [00:14, 11045.18it/s]


In [ ]:
import csv
from tqdm import tqdm

if New_Process_Data:
    with open('test.csv', "r", encoding="utf8") as in_csv, open('processtest.csv', "w", newline="", encoding="utf8") as out_csv:
        read = csv.reader(in_csv)
        write = csv.writer(out_csv)
        next(read, None) # Skip header
        for i in tqdm(read):
            i[1] = processing(i[1])
            try:
                write.writerow(i)
            except Exception as e:
               print(e)

153164it [00:13, 11681.65it/s]


In [ ]:
#df['comment_text'] = df['comment_text'].apply(processing)

#test_df['comment_text'] = test_df['comment_text'].apply(processing)


In [ ]:
print(df['comment_text'][0])

Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27


In [ ]:
#df.to_csv('traindata.csv')
#test_df.to_csv('testdata.csv')

In [ ]:
import torchtext.data as textdata

In [ ]:
text = textdata.Field(batch_first = True,
                  tokenize = spacy_tokenizer,
                  stop_words = stopwords_list)

label = textdata.LabelField(dtype = torch.float)

id1 = textdata.LabelField()

id2 = textdata.Field(sequential=False)

In [ ]:
Features = [["id",id1], ["text", text], ["toxic",label],["severe_toxic",label],
          ["obscene",label],["threat",label],["insult",label],["identity_hate",label]]

Test_Features = [["id",id2], ["text", text]]

train_data = textdata.TabularDataset('processtrain.csv', format = 'csv', fields=Features, skip_header=True)

test_data = textdata.TabularDataset('processtest.csv', format = 'csv', fields=Test_Features, skip_header=False)

In [ ]:
train_data, val_data = train_data.split(split_ratio=0.75,random_state=random.seed(2))

In [ ]:
#Had to try different petrained vectors, as I kept getting weird HTTP errors with some of them.
text.build_vocab(train_data,
                 min_freq = 3,
                 max_size = vocab_Max,
                 vectors = "glove.6B.50d", 
                 unk_init = torch.Tensor.normal_)

label.build_vocab(train_data)

id1.build_vocab(train_data)
id2.build_vocab(test_data)

In [ ]:
vocab_list = (text.vocab.freqs.most_common(10))
print('The 10 most common words in my vocabulary are: ')
for i in range(len(vocab_list)):
  print(str(i+1), ": '", vocab_list[i][0],"'.", "It occured ", vocab_list[i][1], " times!")

The 10 most common words in my vocabulary are: 
1 : ' article '. It occured  43118  times!
2 : ' page '. It occured  34858  times!
3 : ' wikipedia '. It occured  34773  times!
4 : ' talk '. It occured  28026  times!
5 : ' please '. It occured  22280  times!
6 : ' would '. It occured  21951  times!
7 : ' one '. It occured  21849  times!
8 : ' like '. It occured  20953  times!
9 : ' see '. It occured  16164  times!
10 : ' also '. It occured  15563  times!


In [ ]:
print("The number of unique words in the training set is: ", str(len(text.vocab)))
print("The number of unique labels in the training set is: ", str(len(label.vocab)))

The number of unique words in the training set is:  50002
The number of unique labels in the training set is:  2


In [ ]:
batch_size = 64

train_i, val_i = textdata.BucketIterator.splits((train_data, val_data),
                                                  batch_size=batch_size,
                                                  device = device)

test_i = textdata.BucketIterator(test_data,
                                batch_size=batch_size,
                                shuffle=False,
                                device = device)

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, vocab_size, embed_dim, n_filters, filter_sizes, output_dim, 
                 dropout,pad_idx):
        super(ConvNet, self).__init__() 
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.convs = nn.ModuleList([nn.Conv2d(in_channels = 1, out_channels = n_filters, kernel_size = (fs, embed_dim)) for fs in filter_sizes])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        embedded = self.embedding(text)          
        embedded = embedded.unsqueeze(1)
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]       
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved] 
        cat = self.dropout(torch.cat(pooled, dim = 1))
            
        return self.fc(cat)

In [ ]:
input_dim = len(text.vocab)
embed_dim = 100
n_filters = 32
filter_sizes = [2,3,4]
#For the 6 labels we have.
output_dim = 6
dropout = 0.5
pad_idx = text.vocab.stoi[text.pad_token]
#UNK_IDX = text.vocab.stoi[text.unk_token]

model = ConvNet(input_dim, embed_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx)

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 2e-5, weight_decay=1e-4)
criterion = nn.BCEWithLogitsLoss().to(device)
print(model)

ConvNet(
  (embedding): Embedding(50002, 100)
  (convs): ModuleList(
    (0): Conv2d(1, 32, kernel_size=(2, 100), stride=(1, 1))
    (1): Conv2d(1, 32, kernel_size=(3, 100), stride=(1, 1))
    (2): Conv2d(1, 32, kernel_size=(4, 100), stride=(1, 1))
  )
  (fc): Linear(in_features=96, out_features=6, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [ ]:
def get_label(batch):
    toxic = batch.toxic.unsqueeze(1)
    severe_toxic = batch.severe_toxic.unsqueeze(1)
    obscene = batch.obscene.unsqueeze(1)
    threat = batch.threat.unsqueeze(1)
    insult = batch.insult.unsqueeze(1)
    identity_hate = batch.identity_hate.unsqueeze(1)
    labels = torch.cat((toxic,severe_toxic,obscene,
                        threat,insult,identity_hate),dim=1)
    return labels

from tqdm import tqdm

def train_pace(model, optimizer, criterion, batch):
    batch_size = len(batch)
    model.train()
    
    optimizer.zero_grad()
    text = batch.text.view(batch_size, -1)
    labels = get_label(batch)

    outputs = model(text)
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()

    return loss.item()

In [ ]:
#To decide if you want to retrain the model or load a previously saved version.
train_model = True
batch_size = 128
if train_model:
    EPOCHS = 2
    loss_list = []
    print("Training now...")
    for epoch in range(EPOCHS):
        for i, batch in enumerate(train_i):
            train_loss = train_pace(model,optimizer, criterion, batch)
            loss_list.append(train_loss)
            print(f"Epoch: [{epoch+1}/{EPOCHS}] | Iterations: [{i+1}/{len(train_i)}] | Training loss: {train_loss:.3f}")
    torch.save(model.state_dict(), "modelConvNet.pt")
    print("Training done.")


Training now...
Epoch: [1/2] | Iterations: [1/1870] | Training loss: 0.687
Epoch: [1/2] | Iterations: [2/1870] | Training loss: 0.712
Epoch: [1/2] | Iterations: [3/1870] | Training loss: 0.691
Epoch: [1/2] | Iterations: [4/1870] | Training loss: 0.691
Epoch: [1/2] | Iterations: [5/1870] | Training loss: 0.703
Epoch: [1/2] | Iterations: [6/1870] | Training loss: 0.658
Epoch: [1/2] | Iterations: [7/1870] | Training loss: 0.641
Epoch: [1/2] | Iterations: [8/1870] | Training loss: 0.680
Epoch: [1/2] | Iterations: [9/1870] | Training loss: 0.719
Epoch: [1/2] | Iterations: [10/1870] | Training loss: 0.709
Epoch: [1/2] | Iterations: [11/1870] | Training loss: 0.656
Epoch: [1/2] | Iterations: [12/1870] | Training loss: 0.691
Epoch: [1/2] | Iterations: [13/1870] | Training loss: 0.679
Epoch: [1/2] | Iterations: [14/1870] | Training loss: 0.699
Epoch: [1/2] | Iterations: [15/1870] | Training loss: 0.647
Epoch: [1/2] | Iterations: [16/1870] | Training loss: 0.664
Epoch: [1/2] | Iterations: [17/18

In [ ]:
if train_model = False:
    model.load_state_dict(torch.load("modelConvNet.pt"))

In [ ]:
#This function makes predictions of the toxicity level, for all the comments in the test iterator.
def predictions(model, test_i, first_idx, sec_idx):
    result = []
    #put model into evaluation mode, so that the weights are not affected.
    model.eval()
    with torch.no_grad():
        for batch in tqdm(test_i):
            batch_size = len(batch)
            text = batch.text.view(batch_size,-1).long()
            ids = batch.id.squeeze().cpu()
            output = model(text)
            output = torch.sigmoid(output).cpu()*10
            for i,j in zip(ids,output):
                result.append([id2.vocab.itos[i.numpy()],j.numpy()])
    return result[first_idx][sec_idx], text

In [ ]:
tqdm(test_i)

  0%|          | 0/2394 [00:00<?, ?it/s]

  0%|          | 0/2394 [00:00<?, ?it/s]

In [ ]:
results, text_comment = predictions(model, test_i, 4,1)
print(results)
print(text_comment)

100%|██████████| 2394/2394 [03:07<00:00, 12.76it/s]


ValueError: ignored

In [ ]:
minimum_text_len = 5
def pred_toxicity(model, sentence):
    model.eval()
    tokens = [i.text for i in nlp.tokenizer(sentence)]
    if len(tokens) < minimum_text_len:
        tokens += ['<pad>'] * (minimum_text_len - len(tokens))
    indexed = [text.vocab.stoi[w] for w in tokens]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction

In [ ]:
comment = processing('Fuck you, Smith. Please have me notified when you die. I want to dance on your grave.')
pred_toxicity(model,comment)

tensor([[0.2518, 0.2185, 0.1761, 0.0939, 0.1315, 0.0797]],
       grad_fn=<SigmoidBackward>)